# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846736623807                   -0.70    4.5
  2   -7.852303821300       -2.25       -1.53    1.0
  3   -7.852645949830       -3.47       -2.53    3.0
  4   -7.852646678629       -6.14       -3.36    2.8
  5   -7.852646686065       -8.13       -4.80    1.0
  6   -7.852646686725       -9.18       -5.25    4.2
  7   -7.852646686730      -11.32       -6.02    2.0
  8   -7.852646686730      -12.69       -7.53    1.8
  9   -7.852646686730      -15.05       -7.88    2.2
 10   -7.852646686730      -14.57       -8.55    1.0
 11   -7.852646686730   +    -Inf       -9.39    3.2
 12   -7.852646686730   +    -Inf       -9.01    1.0
 13   -7.852646686730   +  -14.27       -8.87    1.0
 14   -7.852646686730      -14.45       -8.81    1.0
 15   -7.852646686730   +  -14.75       -9.44    1.0
 16   -7.852646686730      -14.45       -9.14    1.0
 17   -7.852646686730   +  -14.57      -10.81 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846728065981                   -0.70           4.8
  2   -7.852525173378       -2.24       -1.63   0.80    2.0
  3   -7.852637009199       -3.95       -2.70   0.80    1.0
  4   -7.852646424906       -5.03       -3.30   0.80    2.2
  5   -7.852646677875       -6.60       -4.10   0.80    1.5
  6   -7.852646686477       -8.07       -4.70   0.80    1.5
  7   -7.852646686721       -9.61       -5.61   0.80    1.2
  8   -7.852646686730      -11.06       -6.45   0.80    2.0
  9   -7.852646686730      -12.70       -7.45   0.80    1.8
 10   -7.852646686730      -14.27       -8.48   0.80    2.0
 11   -7.852646686730      -14.75       -9.15   0.80    1.8
 12   -7.852646686730   +  -14.45       -9.73   0.80    1.8
 13   -7.852646686730   +    -Inf      -10.58   0.80    2.2
 14   -7.852646686730   +  -15.05      -11.83   0.80    2.2
 15   -7.852646686730      -15.05      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.382746e+01     3.530947e+00
 * time: 0.5062541961669922
     1     1.173781e+00     1.980115e+00
 * time: 0.71563720703125
     2    -1.642220e+00     2.311241e+00
 * time: 0.7424840927124023
     3    -3.990369e+00     2.018864e+00
 * time: 0.7805020809173584
     4    -5.290798e+00     1.904194e+00
 * time: 0.8184812068939209
     5    -6.905898e+00     1.068103e+00
 * time: 0.8572371006011963
     6    -6.972481e+00     2.186421e+00
 * time: 0.8836960792541504
     7    -7.540171e+00     1.962677e+00
 * time: 0.910006046295166
     8    -7.599818e+00     3.441565e+00
 * time: 0.9362561702728271
     9    -7.664461e+00     1.106168e+00
 * time: 0.9743361473083496
    10    -7.703325e+00     5.990561e-01
 * time: 1.0318682193756104
    11    -7.774785e+00     6.420904e-01
 * time: 1.0585901737213135
    12    -7.796587e+00     8.410452e-02
 * time: 1.0850391387939453
    13    -7.810985e+00     9.301242e-02
 * time: 1.1114380359649

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846862472936                   -0.70    4.5
  2   -7.852315812093       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686680                   -2.55
  2   -7.852646686730      -10.31       -5.87
  3   -7.852646686730   +  -14.45      -12.09


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.49374722865254e-13
|ρ_newton - ρ_scfv| = 2.742461740982124e-13
|ρ_newton - ρ_dm|   = 8.668504826270394e-10
